In [1]:
from z3 import *

## Equality with Uninterpreted Functions

In [ ]:
S = RealSort()

x = Const('x', S)
f = Function('f', S, S)

In [ ]:
solve(f(f(x)) == x, f(f(f(x))) == x, f(x) != x)

Using the first equality $f(f(x)) = x$ we get $f(f(f(x))) = x \mapsto f(x) = x$, but $f(x) \neq x$.

In [ ]:
solve(f(f(x)) == x, f(f(f(x))) == x)

In [ ]:
y = Const('y', S)
g = Function('g', S, S, S)
solve(g(x,y) == x, g(g(x,y),y) != x)

Using the first equality $g(x,y) = x$ we get $g(g(x,y),y) \mapsto g(x,y)= x$, but $g(x,y)\neq x$.

In [ ]:
solve(g(y,x) == x, g(g(x,y),y) != x)

Types are important

In [ ]:
x = Int("x")
solve(x * x == 2)

In [ ]:
x = Real("x")
solve(x * x == 2)

## Arithmetic

In [ ]:
a, b, c = Ints("a b c")
d, e = Reals("d e")

s = Solver()
s.add(a < b + 2)
s.add(a == 2 * c + 10)
s.add(b + c == 1000)
s.add(d == e)

s.check()
s.model()

In [ ]:
s.reset()

s.add(e < a + b + 2.0)
s.add(d == c + 0.5)

s.check()
s.model()

In [ ]:
s.reset()

# s.add(a < b < c) not allowed
s.add(a < b)
s.add(b < c)

s.check()
s.model()

In [ ]:
# how do reals behave under rounding?
# sum of two reals xR, yR is below a
# sum of round(xR), round(yR) is above a
# rounded numbers are within unit distance to the reals

s.reset()

x, y, a = Ints("x, y, a")
xR, yR = Reals("xR, yR")

s.add(xR + yR < a)
s.add(x + y > a)

s.add(Or(x == xR, And(x < xR, xR < x + 1), And(x - 1 < xR, xR < x)))
s.add(Or(y == yR, And(y < yR, yR < y + 1), And(y - 1 < yR, yR < y)))

s.check()
s.model()

In [ ]:
# solving simple job-shop scheduling constraints using difference arithmetic
# tasks have start time, duration, and constraints (specifying whether tasks should not overlap and ordering)
# to specify the constraints, we compare two time points separated by a constant offset

s.reset()

t11, t12, t21, t22, t31, t32 = Ints("t11 t12 t21 t22 t31 t32")

s.add(And(t11 >= 0, t12 >= t11 + 2, t12 + 1 <= 8))
s.add(And(t21 >= 0, t22 >= t21 + 3, t22 + 1 <= 8))
s.add(And(t31 >= 0, t32 >= t31 + 2, t32 + 3 <= 8))
s.add(Or(t11 >= t21 + 3, t21 >= t11 + 2))
s.add(Or(t11 >= t31 + 2, t31 >= t11 + 2))
s.add(Or(t21 >= t31 + 2, t31 >= t21 + 3))
s.add(Or(t12 >= t22 + 1, t22 >= t12 + 1))
s.add(Or(t12 >= t32 + 3, t32 >= t12 + 1))
s.add(Or(t22 >= t32 + 3, t32 >= t22 + 1))

s.check()
s.model()

In [ ]:
# nonlinear arithmetic: very expressive, undecidable
# the solver does not always find solutions to nonlinear problems

s.reset()

x = Real("x")
s.add(2 ** x == 3)

s.check()
#s.model()

In [ ]:
# yet it can show unsatisfiability for some nontrivial nonlinear problems

s.reset()

x, y, z = Reals("x, y, z")
s.add(x * x == x + 2.0)
s.add(x * y == x)
s.add((y - 1.0) * z / x == 1.0)

s.check()
#s.model()

In [ ]:
# when presented only nonlinear polynomial constraints over reals, z3 uses a specialized complete solver

s.reset()

b, c = Reals("b, c")
s.add(b * b * b + b * c == 3.0)

s.check()
s.model()

## Arrays

In [ ]:
Z = IntSort()
A = Array('A', Z, Z)
x, y = Ints('x y')

s = Solver()
s.add(A[x] == x, Store(A, x, y) == A) 
# Sets x to be value of A at position x; Stores y at position x.
s.check()
s.model()

$K(Int, 2)$ is the constant array containing $2$ at every entry; $ Store(K(Int, 2), 3, 3)$ indicates  that $3$ is added at position $3$. 

In [ ]:
s.reset()

s.add(A[x] == x, Store(A, x, y) == A, x != y)

s.check()
#s.model()

Because $x = A[x]$ and $A[x]=y$

## Groups

In [ ]:
G = DeclareSort("G")
e,a,b,c = Consts("e a b c", G)
times = Function('*', G,G, G)
inv = Function('~', G,G)

axioms = [
  ForAll([a], times(inv(a),a) == e ), # inverse
  ForAll([a], times(a,inv(a)) == e ),
  ForAll([a], times(e,a) == a ), # identity
  ForAll([a], times(a,e) == a ),
  ForAll([a,b,c] , times(a,times(b,c)) == times(times(a,b),c)) # associativity
]

# A simple theorem
s = Solver()
s.add(axioms)
theorem = inv(times(a,b)) == times(inv(b), inv(a))
s.add(Not(theorem))
s.check()

In [ ]:
G = DeclareSort("G")
e,a,b,c = Consts("e a b c", G)
times = Function('*', G,G, G)
inv = Function('~', G,G)

axioms = [
  ForAll([a], times(inv(a),a) == e ), # inverse
  ForAll([a], times(a,inv(a)) == e ),
  ForAll([a], times(e,a) == a ), # identity
  ForAll([a], times(a,e) == a ),
  ForAll([a,b,c] , times(a,times(b,c)) == times(times(a,b),c)) # associativity
]

# A simple theorem
s = Solver()
s.add(axioms)
theorem = inv(times(a,b)) == times(inv(b), inv(a))
s.add(theorem)
s.check()

$(a\cdot b)^{-1}=a^{-1}\cdot b^{-1}$

## Induction

In [2]:
n = Int("n")
Sumn = Function("sumn", IntSort(), IntSort())
s = Solver()

# defining sum(n) = 1 + 2 + 3 + ... n
s.add( Sumn(0) == 0)
s.add(ForAll([n], Sumn(n+1) == n + 1 + Sumn(n))) 

s.add(Not(ForAll([n], Implies(n >= 0, 2 * Sumn(n) == n * (n + 1)))))
s.check()

unknown

In [3]:
n = Int("n")
Sumn = Function("sumn", IntSort(), IntSort())
s = Solver()

# defining sum(n) = 1 + 2 + 3 + ... n
s.add( Sumn(0) == 0)
s.add(ForAll([n], Sumn(n+1) == n + 1 + Sumn(n)))

def prop_sumn(n):
    return 2 * Sumn(n) == n * (n + 1)

def induction(p):
    n = Int("n")
    return Implies(And(  p(0),    ForAll([n],Implies(And(n >= 0, p(n)),p(n+1)))),
                     # --------------------------------------------------------
                       ForAll( [n] , Implies(n>=0, p(n))))

s.add(induction(prop_sumn))

s.add( Not(ForAll([n], Implies( n >= 0, prop_sumn(n)))))
if s.check() == unsat:
    print("proved")

proved


$(p(0) \land \forall n . (n>=0 \land p(n)) \implies p(n+1))) \implies \forall n. (n>=0 \implies p(n))$

## An Application: Install Problem
https://ericpony.github.io/z3py-tutorial/guide-examples.htm

The install problem consists of determining whether a new set of packages can be installed in a system. This application is based on the article OPIUM: Optimal Package Install/Uninstall Manager. Many packages depend on other packages to provide some functionality. Each distribution contains a meta-data file that explicates the requirements of each package of the distribution. The meta-data contains details like the name, version, etc. More importantly, it contains depends and conflicts clauses that stipulate which other packages should be on the system. The depends clauses stipulate which other packages must be present. The conflicts clauses stipulate which other packages must not be present.

In [4]:
def DependsOn(pack, deps):
    if is_expr(deps):
        return Implies(pack, deps)
    else:
        return And([ Implies(pack, dep) for dep in deps ])

def Conflict(*packs):
    return Or([ Not(pack) for pack in packs ])

a, b, c, d, e, f, g, z = Bools('a b c d e f g z')

def install_check(*problem):
    s = Solver()
    s.add(*problem)
    if s.check() == sat:
        m = s.model()
        r = []
        for x in m:
            if is_true(m[x]):
                # x is a Z3 declaration
                # x() returns the Z3 expression
                # x.name() returns a string
                r.append(x())
        print (r)
    else:
        print ("invalid installation profile")

print ("Check 1")
install_check(DependsOn(a, [b, c, z]),
              DependsOn(b, d),
              DependsOn(c, [Or(d, e), Or(f, g)]),
              Conflict(d, e),
              Conflict(d, g),
              a, z)

print ("Check 2")
install_check(DependsOn(a, [b, c, z]),
              DependsOn(b, d),
              DependsOn(c, [Or(d, e), Or(f, g)]),
              Conflict(d, e),
              Conflict(d, g),
              a, z, g)


Check 1
[z, b, d, f, c, a]
Check 2
invalid installation profile


## An Application: Propositional Interpolation
https://theory.stanford.edu/~nikolaj/programmingz3.html#sec-propositional-interpolation


In [ ]:
a1, a2, b1, b2, x1, x2 = Bools('a1 a2 b1 b2 x1 x2')
A = And(a1 == x1, a2 != a1, a2 != x2)
B = And(b1 == x1, b2 != b1, b2 == x2)
s = Solver()

s.add(A)
s.add(B)

s.check()

In [ ]:
def mk_lit(m, x):
    if is_true(m.eval(x)):
        return x
    else:
        return Not(x)

def pogo(A, B, xs):   
    while sat == A.check():
        m = A.model()
        print("Model", m)
        L = [mk_lit(m, x) for x in xs]
        print("literals", L)
        if unsat == B.check(L):
            notL = Not(And(B.unsat_core()))
            print("notL:", notL)
            yield notL
            A.add(notL)
        else:
            print("expecting unsat")
            break

A = SolverFor("QF_FD")
B = SolverFor("QF_FD")
a1, a2, b1, b2, x1, x2 = Bools('a1 a2 b1 b2 x1 x2')
A.add(a1 == x1, a2 != a1, a2 != x2)
B.add(b1 == x1, b2 != b1, b2 == x2)
print(list(pogo(A, B, [x1, x2])))

## An Application: Bounded Model Checking
https://theory.stanford.edu/~nikolaj/programmingz3.html#sec-bounded-model-checking

In [ ]:
index = 0
def fresh(round, s):
    global index
    index += 1
    return Const("!f%d_%d" % (round, index), s)

def zipp(xs, ys):
    return [p for p in zip(xs, ys)]

def bmc(init, trans, goal, fvs, xs, xns):
    s = Solver()
    s.add(init)
    count = 0
    while True:
        print("iteration ", count)
        count += 1
        p = fresh(count, BoolSort())
        s.add(Implies(p, goal))
        if sat == s.check(p):
            print (s.model())
            return
        s.add(trans)
        ys = [fresh(count, x.sort()) for x in xs]
        nfvs = [fresh(count, x.sort()) for x in fvs]
        trans = substitute(trans, 
                           zipp(xns + xs + fvs, ys + xns + nfvs))
        goal = substitute(goal, zipp(xs, xns))
        xs, xns, fvs = xns, ys, nfvs


if __name__ == '__main__':
    x0, x1 = Consts('x0 x1', BitVecSort(4))
    bmc(x0 == 0, x1 == x0 + 3, x0 == 10 , [], [x0], [x1])